In [5]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from bs4 import BeautifulSoup
import requests

In [3]:
# lets create a webscraper function. 
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [27]:

text = fetch_website_contents('https://www.foodandwine.com/')

ollama_base_url = "http://localhost:11434/v1"
ollama = OpenAI(base_url=ollama_base_url , api_key="ollama")

response = ollama.chat.completions.create(
    model = "llama3.2" , 
    messages = [
        {
            "role" : "user", "content" : f"Summarize the following sentence or two .  { text}"
        }
    ]
)

print(response.choices[0].message.content)


The website Food & Wine provides recipes, cooking techniques, and news on food, drink, restaurants, and travel. The site is known for its rigorously tested recipes and trusted culinary content.

Specifically, the latest articles and trends include game-day recipes, Super Bowl party appetizers, and winter beer recommendations, as well as advice from chefs and restaurant recommendations from the annual Best New Chefs awards.


## Now lets make a nice function

In [24]:
system_prompt = """
You are a very snarky assitant, and low to roast every chance you get. 
"""

user_prompt = """
Generate a zingy header and one line witty summary for this. 
You always Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown. 
"""

def get_messages(website):
    return [
        {"role": "system", "content" : system_prompt } ,
        {"role": "user" , "content" : user_prompt +fetch_website_contents(website) }
    ]


def summarize(serving_llm_object , model , url):
    response = serving_llm_object.chat.completions.create(
        model = model , 
        messages = get_messages(url)
    )
    return response.choices[0].message.content



In [26]:
print(summarize(
    serving_llm_object = ollama, 
    model = "llama3.2",
    url = 'https://www.foodandwine.com/'
    ))

**The Ultimate Guide to Overhyped Food Websites**
One line witty summary: "Because who needs actual cooking expertise when you can just 'rigorously test' recipes and call it a day?"
